In [1]:
# On importe les librairies usuelless
import math
import numpy as np
import matplotlib.pyplot as plt

# On désactive les warnings
import warnings
warnings.filterwarnings('ignore')

In [18]:
from tensorflow import keras
from tensorflow.keras.layers import Dense , Input , SimpleRNN, LSTM , Embedding, Dropout
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Convolution1D
from tensorflow.keras import layers
from keras.layers import Lambda
import keras.backend as K

In [3]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

In [4]:
def ctc_complete_decoding_lambda_func(args, **arguments):
    """
    Complete CTC decoding using Keras (function K.ctc_decode)
    :param args: 
        y_pred, input_length
    :param arguments:
        greedy, beam_width, top_paths
    :return: 
        K.ctc_decode with dtype='float32'
    """

    # import tensorflow as tf # Require for loading a model saved

    y_pred, input_length = args
    my_params = arguments

    assert (K.backend() == 'tensorflow')

    return K.cast(K.ctc_decode(y_pred, tf.squeeze(input_length), greedy=my_params['greedy'],
                               beam_width=my_params['beam_width'], top_paths=my_params['top_paths'])[0][0],
                  dtype='float32')


In [5]:
embed_size = 128                                                   # dimension de l'embedding
LSTM_size   = 500
# Example d'architecture pour le réseau RNN simple avec dropout

In [10]:
labels = Input(name='labels', shape=[None])
input_length = Input(name='input_length', shape=[1])
label_length = Input(name='label_length', shape=[1])

input_data = Input(name='input', shape=(None, 128)) # nb_features = image height

blstm = Bidirectional(LSTM(LSTM_size, return_sequences=True))(input_data)
blstm = Bidirectional(LSTM(LSTM_size, return_sequences=True))(blstm)
blstm = Bidirectional(LSTM(LSTM_size, return_sequences=True))(blstm)
blstm = Bidirectional(LSTM(LSTM_size, return_sequences=True))(blstm)
blstm = Bidirectional(LSTM(LSTM_size, return_sequences=True))(blstm)

In [11]:
loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc') ([blstm, labels, input_length, label_length])

In [19]:
model = Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer  = keras.optimizers.SGD(learning_rate=10**(-4),momentum=0.9))
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, None, 128)]  0           []                               
                                                                                                  
 bidirectional_5 (Bidirectional  (None, None, 1000)  2516000     ['input[0][0]']                  
 )                                                                                                
                                                                                                  
 bidirectional_6 (Bidirectional  (None, None, 1000)  6004000     ['bidirectional_5[0][0]']        
 )                                                                                                
                                                                                            

In [ ]:
model.fit_generator(generator=training_generator,
                    use_multiprocessing=True,
                    workers=2)
model.save('saved_model/my_model')

In [ ]:
test = 
prediction = model.predict(test)

# use CTC decoder
out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1],greedy=True)[0][0]).predict(test)